In [ ]:
import os
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine, text
import time
import numpy as np
import dotenv

dotenv.load_dotenv(".env.local")

from test_unit.tasks_unit1 import UnitTask1
from mrp_functions.base_tasks import TaskBase
from mrp_functions import logging
import re

np.__version__

part_group=os.environ['PART_GROUP']
contract_code=os.environ['CONTRACT_CODE']
mrp_run_date=os.environ['MRP_RUN_DATE']
msc_code=os.environ['MSC_CODE']
plant_code=os.environ['PLANT_CODE']
production_plan_id=int(os.environ['PRODUCTION_PLAN_ID'])
user_code=int(os.environ['USER_CODE'])
simulation=bool(int(os.environ['SIMULATION']))

tb = TaskBase()
tb.db_connection_health_check()

In [ ]:
# Get end date of production plan
end_date_of_prod_plan = tb.query_end_date_from_production_plan(production_plan_id)
if end_date_of_prod_plan is None:
    raise ValueError("Production Plan End Date is None!")
# Get list of all date from N to N+6 to end of production Plan
list_mrp_date = tb.query_list_date_from_Nplus1_to_Nplus6_definition(mrp_run_date, end_date_of_prod_plan)
print(f"Inventory Forecast From: {list_mrp_date[0]} To: {list_mrp_date[-1]}")

In [ ]:
# To query current Logical Inventory Quantity
log_inventory_frame = tb.query_inventory_log(mrp_run_date, plant_code, simulation)
logging.info(f"[{production_plan_id}_{mrp_run_date}_{plant_code}] "
             f"Current Inventory Size: {len(log_inventory_frame)}")
log_inventory_frame = log_inventory_frame.pivot_table(
            index="production_date",
            columns=["part_code", "part_color_code"],
            values="quantity",
            aggfunc='sum')
log_inventory_frame

In [ ]:
mrp_results_frame = tb.query_mrp_results(mrp_run_date, plant_code, simulation)
logging.info(f"[{production_plan_id}_{mrp_run_date}_{plant_code}] "
             f"MRP Results Size: {len(mrp_results_frame)}")
mrp_results_frame = mrp_results_frame.pivot_table(
            index="production_date",
            columns=["part_code", "part_color_code"],
            values="quantity",
            aggfunc='sum')
mrp_results_frame

In [ ]:
merge_logical_parts_columns = mrp_results_frame.columns.tolist()
for _col in log_inventory_frame.columns:
    if _col not in merge_logical_parts_columns:
        merge_logical_parts_columns.append(_col)

merge_logical_parts_inv = pd.DataFrame(np.zeros((len(list_mrp_date), len(merge_logical_parts_columns))), index=list_mrp_date)
merge_logical_parts_inv.index.names = ['production_date']
merge_logical_parts_inv.columns = pd.MultiIndex.from_tuples(merge_logical_parts_columns, names=['part_code', 'part_color_code'])

merge_mrp_result_parts = (merge_logical_parts_inv + mrp_results_frame.fillna(0.0)).fillna(0.0)

if len(log_inventory_frame) == 0:
    merge_logical_parts_inv = merge_logical_parts_inv.fillna(0.0)
else:
    merge_logical_parts_inv = (merge_logical_parts_inv + log_inventory_frame).fillna(0.0)

merge_logical_parts_inv


In [ ]:
final_logical_inventory = merge_logical_parts_inv.cumsum() - merge_mrp_result_parts.cumsum()
final_logical_inventory


In [ ]:
final_shortage_parts = merge_logical_parts_inv.cumsum() - merge_mrp_result_parts.cumsum()
final_shortage_parts = final_shortage_parts.applymap(lambda x: np.minimum(0.0, x))
final_shortage_parts = final_shortage_parts - final_shortage_parts.shift(1).copy(deep=True).fillna(0.0)
final_shortage_parts
